In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from qaoa_and_tdvp import QAOAResult
import math
from scipy.interpolate import griddata
import numpy as np
from qaoa_and_tdvp import QAOA

In [ ]:
with open(f'.//results//data.p', 'rb') as file:
    data = pickle.load(file)
data.head(6)

In [ ]:
with open(f'.//results//stats.p', 'rb') as file:
    stats = pickle.load(file)
stats

In [ ]:
stats['expectation approximation ratio'].reset_index()

In [ ]:
def algo_num(algo:str)->int:
    assert algo in {'tdvp','scipy','gradient_descent'}
    if algo=='tdvp':
        return 1
    elif algo=='scipy':
        return 0
    elif algo=='gradient_descent':
        return -1

In [ ]:
BA_template = dict(
    layout=go.Layout(
    template='plotly_white',
    yaxis=dict(
        range = [0,1.1],
        tickmode = 'linear',
        tick0 = 0,
        dtick = .2,
        autorange=False,
        linewidth = 1,
        linecolor = 'black',
        gridcolor = 'lightgrey',
        zerolinecolor = 'black',

    ),
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1,
        range=[-.5,5],
        autorange=True,
        # type='category',
        linewidth = 1,
        linecolor = 'black',
        gridcolor = 'lightgrey',
        # zerolinecolor = 'black',

    ),
    font=dict(
        family='Computer Modern',
        size=11,
    ),
    legend=dict(
        yanchor="bottom",
        y=0.01,
        xanchor="right",
        x=.99,
        
    ),
    # paper_bgcolor='rgba(0,0,0,0)',   # transparent background
    plot_bgcolor='rgba(0,0,0,0)',)
)

In [ ]:
stats.xs(3, level='n')

In [ ]:
meas = 'expectation approximation ratio'
n=3
df = stats[meas].xs(3, level='n').reset_index()
df

In [ ]:
#flattend_data
data_red = pd.concat([
    data['tdvp'].xs(n,level='n').apply(lambda x: x),
    data['scipy'].xs(n,level='n').apply(lambda x:x),
    data['gradient_descent'].xs(n,level='n').apply(lambda x:x)
]).reset_index(drop=True)
data_red

In [ ]:


# TODO: add different symbols for different algorithms (colorblind friendly), 
# TODO: make scatter plot nice (transparency, etc.)

gridcolor = 'lightgrey'



scatter = px.scatter(
    data_frame=df,
    x=df['p']+.1*df['Algorithm'].apply(algo_num), # for the offset
    y='mean',
    error_y='std',
    color='Algorithm',
    symbol='Algorithm',
    # barmode='group',
)


box = px.box(
    data_frame=data_red,
    x='p',
    y=meas,
    color='algorithm',
    # points='all',
)
box.update_traces(quartilemethod="inclusive") # or "inclusive", or "linear" by default

fig = box

# rename traces 
newnames = {'tdvp':'TDVP', 'scipy':'COBYLA', 'gradient_descent':'GD'}
fig.for_each_trace(lambda t: t.update(name = newnames.get(t.name,''),
                                      legendgroup = newnames.get(t.name,''),
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

fig.update_layout(
    template=BA_template,
    xaxis_title='p',
    yaxis_title="Expectation of the approximation ratio",
    legend_title_text='Algorithm',
    width= 700,
)

fig.show()

In [ ]:
with open("./results/grid_search_n5_p2_i5.p",'rb') as file:
    grid_search = pickle.load(file)

In [ ]:
with open("results/data.p",'rb') as file:
    data = pickle.load(file)

In [ ]:
idx = (5,1,5)
qaoa = QAOA(data.loc[idx]['instance'],p=idx[1])

In [ ]:
tdvp_path = data.loc[idx]['tdvp']['path']
tdvp_x = np.array([d[0] for d in tdvp_path])
tdvp_y = np.array([d[1] for d in tdvp_path])
tdvp_z = np.array([qaoa.expectation(d) for d in tdvp_path])

In [ ]:
grid_search.max()

In [ ]:
Zlandscape.shape

In [ ]:
griddata((tdvp_x,tdvp_y),tdvp_z,(xi,yi))

In [ ]:

df = grid_search

x = np.array(df.beta)
y = np.array(df.gamma)
z = np.array(df.value)


xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)

X,Y = np.meshgrid(xi,yi)

Zlandscape = griddata((x,y),z,(X,Y), method='cubic')


Zpath_tdvp = griddata((tdvp_x,tdvp_y),tdvp_z,(xi,yi))

fig = go.Figure(
    go.Surface(
        x=xi,
        y=yi,
        z=Zlandscape
        ),
    # go.Scatter3d(
    #     x=tdvp_x,
    #     y=tdvp_y,
    #     z=tdvp_z,
    #     mode='markers'
    # )
    )

fig.update_layout(
    template=BA_template,
    width=1000,
    height=1000
)
fig.show()
fig.write_html('./plots/grid_seach_n5_p1_i5_landscape.html')